In [2]:
import requests as rq
from urllib.parse import quote
import pandas as pd
import re
import time
import random
import requests as rq
import bs4
import json
from urllib.parse import quote
import json_repair 
import time
import random
from fake_useragent import UserAgent
import os


In [4]:
class PhoneSpecsHandler:
    """
    A class that handles the retrieval of phone specifications using an API.

    Attributes:
        api (str): The URL of the API.
        headers (dict): The headers to be used for the requests.
        filename (str): The name of the file to save the retrieved data.

    Methods:
        get_phone_brands: Retrieves the available phone brands from the API.
        get_phone_models: Retrieves the phone models for a specific brand from the API.
        get_phone_specs: Retrieves the specifications of a specific phone from the API.
        update_all_data: Retrieves the specifications of all phones from the API.
        save_data: Saves the retrieved phone specifications to a JSON file.
    """

    def __init__(self, filename='phone_data.json'):
        self.api = 'https://phone-specs-api.vercel.app'
        self.headers = {'Content-Type': 'application/json',
                        'User-Agent': 'Mozilla/5.0'}
        self.filename = filename

        try:
            with open(self.filename, 'r') as f:
                self.data = json.load(f)
        except Exception as e:
            print(f'[Error: {e}]: Creating new file...')
            self.data = dict()

            # save the file
            self.save_data()


        

    def __get(self, url):
        """
        Retrieves the JSON response from the API.

        Args:
            url (str): The URL to be requested.

        Returns:
            dict: The JSON response from the API.
        """

        # throttle the requests to avoid getting blocked
        # add random delay between 0.5 and 2 seconds
        start = time.time()
        response = rq.get(url, headers=self.headers, timeout=10).json()
        end = time.time()

        duration = end - start
        if duration < 2:
            d = int((2 - duration)*1000)
            time.sleep(random.randint(d, d + 500)/1000)

        return response
    
    def __try_get(self, url):
        retries = 0
        while True:
            if retries == 5:
                print(f'[Error: {result["error"]}]: Retries exceeded. Skipping...')
                return None
            
            try:
                result = self.__get(url)

                if str(result['status']) == 'True':
                    return result
                elif 'Please provide a valid phone slug!' in result['error']:                
                    print(f'[Error: {result["error"]}]: Retrying in 5s...')
                    retries += 1
                    time.sleep(5)

            except Exception as e:
                print(f'[Error: {e.args[0]}]: Retrying in 5s...')
                retries += 1
                time.sleep(5)                


    def get_phone_brands(self):
        """
        Retrieves the available phone brands from the API.

        Returns:
            dict: The JSON response containing the available phone brands.
        """
        url = self.api + '/brands'
        response = self.__try_get(url)
        
        return response

    def get_phone_models(self, brand_name, brand_id):
        """
        Retrieves the phone models for a specific brand from the API.

        Args:
            brand_name (str): The name of the phone brand.
            brand_id (int): The ID of the phone brand.

        Returns:
            dict: The JSON response containing the phone models for the specified brand.
        """
        url = self.api + f'/brands/{quote(brand_name.lower().replace(" ", "_"))}-phones-{brand_id}'
        phones = self.__try_get(url)
        
        print(phones, url, sep='\n')

        last_page = int(phones['data']['last_page']) 

        print(f'last page: {last_page}')
        if last_page == 1:
            return phones

        for page in range(1, last_page):
            url_page = self.api + f'/brands/{quote(brand_name.lower().replace(" ", "_"))}-phones-f-{brand_id}-0-p{page+1}'
            response_page = self.__try_get(url_page)

            print(phones)

            phones['data']['phones'].extend(response_page['data']['phones'])

        phones['data'].pop('current_page')
        phones['data'].pop('last_page')

        return phones
    
    def get_phone_models(self, brand_json):
            """
            Retrieves the phone models for a given brand from the GSMArena API.

            Args:
                brand_json (dict): The JSON response containing the brand information.

            Returns:
                dict: The updated brand JSON response with the phone models.

            """

            phones = self.__try_get(brand_json['detail'])          

            # Use re.search to find the first match
            match = re.search(r"(.*?)-(\d+)", brand_json['brand_slug'])

            last_page = int(phones['data']['last_page']) 

            if match:
                # Extract the groups from the match
                first_part = match.group(1)  # Text before the last hyphen
                number = int(match.group(2))  # Number after the last hyphen

                print(f'Page 1 out of {last_page} for {brand_json["brand_name"]} is processing.')

                for page in range(1, last_page):
                    url_page = self.api + f'/brands/{quote(first_part)}-f-{number}-0-p{page+1}'
                    response_page = self.__try_get(url_page)

                    if response_page:
                        phones['data']['phones'].extend(response_page['data']['phones'])
                        print(f'Page {page+1} out of {last_page} done.')
                    else:
                        print(f'Page {page+1} out of {last_page} failed to process. Skipping...')


            phones['data'].pop('current_page')
            phones['data'].pop('last_page')

            return phones
    
    def get_phone_specs(self, phone_name):
        """
        Retrieves the specifications of a specific phone from the API.

        Args:
            phone_name (str): The name of the phone.

        Returns:
            dict: The JSON response containing the specifications of the specified phone.
        """
        url = self.api + f'/{quote(phone_name)}'
        response = self.__try_get(url)
        
        return response
    
    def get_result(self, phone_name):
        """
        Retrieves the search results for a specific phone from the API.

        Args:
            phone_name (str): The name of the phone to search for.

        Returns:
            dict: The JSON response containing the search results for the specified phone.
        """
        url = self.api + f'/search?query={quote(phone_name)}'
        response = self.__try_get(url)

        return response
    
    def update_all_data(self, forced=False):
        print('Phone specs scraping at 2s delay per request. Please wait...')

        if forced:
            self.data = dict()

        # brands = self.get_phone_brands()['data'][8:9]
        brands = self.get_phone_brands()['data']


        phone_sum =  sum([int(x) for x in [brand['device_count'] for brand in brands]])

        
        print(f'There are {phone_sum - self.count_data()} phones to be processed.\nNew phones catalog count: {phone_sum}\nOld phones catalog count: {self.count_data()}')

        if phone_sum-self.count_data() == 0:
            brands_len += 1
            print(f'[100% done]: No new phones to be processed. Done processing.')
            return self.data

        brands_len = len(brands)

        current_phone = 0
        total_time = 0
        
        for i, models in enumerate(brands):
            if self.data.get(models["brand_name"]) is None:
                self.data[models["brand_name"]] = {}

            if int(models['device_count']) == len(self.data[models["brand_name"]]):
                current_phone += int(models['device_count'])
                print(f'[{round(((current_phone)/phone_sum)*100, 4): 0.4f}% done; {current_phone}/{phone_sum} phones]: No new {models["brand_name"]} models to be processed. {models["brand_name"]} done processing, {brands_len - i} brands left.')
                continue

            phones = self.get_phone_models(models)['data']['phones']
            phones_len = len(phones)
            

            for j, phone in enumerate(phones):
                if phone['slug'] in self.data[models["brand_name"]].keys(): 
                    if self.data[models["brand_name"]][phone['slug']]['specifications']['Launch']['Status'][0].lower().find("coming soon") != -1:
                        print(f'[COMING SOON]: Trying to update {phone["phone_name"]}...')
                    else:
                        print(f'[{round(((current_phone := current_phone + 1)/phone_sum)*100, 4): 0.4f}% done; {j+1}/{phones_len} {models["brand_name"]} phones; {current_phone}/{phone_sum} phones]: {phone["phone_name"]} already exists. Skipping...')
                        continue


                # initial time
                start = time.time()
                
                specs = self.__try_get(phone['detail'])     
                if specs:        
                    self.data[models["brand_name"]][phone['slug']] = specs['data']
                    # clean all phones' specs data
                    self.data[models["brand_name"]][phone['slug']]['specifications'] = {j['title']: {k['key']: k['val'] for k in j['specs']} for j in  specs['data']['specifications']}

                    # end time
                    end = time.time()
                    remaining_time = ((total_time := total_time + end - start) / (j + 1)) * (phone_sum - (j + 1))
                    eta = time.time() + remaining_time

                    # convert hours to hours and minutes and seconds
                    print(f'[{round(((current_phone := current_phone + 1)/phone_sum)*100, 4): 0.4f}% done; ETA: {time.strftime("%I:%M:%S %p", time.localtime(eta))}, Remaining Time: {time.strftime("%H:%M:%S", time.gmtime(remaining_time))}; {j+1}/{phones_len} {models["brand_name"]} phones; {end-start:.3f}s; {current_phone}/{phone_sum} phones]: {phone["phone_name"]} done processing.')                
                else:
                    print(f'[{round(((current_phone := current_phone + 1)/phone_sum)*100, 4): 0.4f}% done; {j+1}/{phones_len} {models["brand_name"]} phones; {current_phone}/{phone_sum} phones]: {phone["phone_name"]} failed to process. Skipping...')


            print(f'[{round(((current_phone)/phone_sum)*100, 4): 0.4f}% done; {j+1}/{phones_len} {models["brand_name"]} phones; {current_phone}/{phone_sum} phones]: {models["brand_name"]} done processing, {brands_len - i} brands left.')

            # process/ normalize the data

        print(f'[100% done; {current_phone}/{phone_sum} phones]: Done processing.')


        return self.data
    

    def count_data(self):
        return sum([len(self.data[brand]) for brand in self.data])
            

    def save_data(self, filename = 'phone_data.json', forced_normalize = False):
        # open the previous file and store it to a variable

        __temp_data = dict()
        print('Loading the previous data...')
        with open(self.filename, 'r') as f:
            __temp_data = json.load(f)

        print('Processing the data...')
        

        for brand in __temp_data:
            for phone in __temp_data[brand]:
                if phone not in self.data[brand] or forced_normalize:
                    try:
                        # Normalize the announced date
                        self.data[brand][phone]['specifications']['Launch']['Announced'] = self.parse_date(self.data[brand][phone]['specifications']['Launch']['Announced'][0])
                    except Exception as e:
                        pass

                    try:
                        # Normalize the models, seperated by comma
                        if 'Misc' not in self.data[brand][phone]['specifications']:
                            self.data[brand][phone]['specifications']['Misc'] = {'Models': ['Unknown']}
                            continue
                        if 'Models' not in self.data[brand][phone]['specifications']['Misc']:
                            self.data[brand][phone]['specifications']['Misc']['Models'] = ['Unknown']
                            continue
                        self.data[brand][phone]['specifications']['Misc']['Models'] = self.data[brand][phone]['specifications']['Misc']['Models'][0].split(', ')
                    except Exception as e:
                        print(f'[Error: {e}]: {brand} {phone} models normalization failed.')
                        pass

                    try:
                        # Normalize display size
                        if 'Display' not in self.data[brand][phone]['specifications']:
                            self.data[brand][phone]['specifications']['Display'] = {'Size': ['Unknown']}
                            continue

                        if 'Size' not in self.data[brand][phone]['specifications']['Display']:
                            self.data[brand][phone]['specifications']['Display']['Size'] = ['Unknown']
                            continue
                        self.data[brand][phone]['specifications']['Display']['Size'] = self.data[brand][phone]['specifications']['Display']['Size'][0].split(', ')
                    except Exception as e:
                        print(f'[Error: {e}]: {brand} {phone} display size normalization failed.')
                        pass
                    
                    try:
                        # Normalize the battery
                        if 'Battery' not in self.data[brand][phone]['specifications']:
                            self.data[brand][phone]['specifications']['Battery'] = {'Type': ['Unknown']}
                            continue

                        if 'Type' not in self.data[brand][phone]['specifications']['Display']:
                            self.data[brand][phone]['specifications']['Battery']['Type'] = ['Unknown']
                            continue

                        self.data[brand][phone]['specifications']['Battery']['Type'] = self.data[brand][phone]['specifications']['Battery']['Type'][0].split(', ')
                    except Exception as e:
                        print(f'[Error: {e}]: {brand} {phone} battery type normalization failed.')
                        pass


        print('Saving the data...')
        with open(filename, 'w') as f:
            json.dump(self.data, f, indent=1, sort_keys=True)

        # reload the file
        with open(filename, 'r') as f:
            self.data = json.load(f)


    # tools
    @staticmethod
    def parse_date(line):
        # Define separate regex patterns for different formats
        regex_month = r"(\d{4}), (\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\b)(?:.*)"
        regex_quarter = r"(\d{4}), Q(\d)(?:.*)"
        regex_year_only = r"(\d{4})(?:.*)"

        # Match based on month presence
        if match_month := re.search(regex_month, line):
            year, month = match_month.groups()
            return year, month
        # Match based on quarter presence
        elif match_quarter := re.search(regex_quarter, line):
            year, quarter = match_quarter.groups()
            month_map = {1: "January", 2: "April", 3: "July", 4: "October"}
            month = month_map[int(quarter)]
            return year, month
        # Match based on year only
        elif match_year := re.search(regex_year_only, line):
            year = match_year.group(1)
            return year, "January"
        else:
            return []



In [5]:
print("Initialising the phone data...")

handler = PhoneSpecsHandler()
#brands = handler.get_phone_brands()

print("Phone data initialised.")

Initialising the phone data...
Phone data initialised.


In [50]:
%%script echo skipping

handler.save_data(forced_normalize=True)

Couldn't find program: 'echo'


In [6]:
handler.count_data()

12715

In [ ]:
data = handler.update_all_data()

# save the data to a JSON file
handler.save_data()
print('Done.')

In [52]:
%%script echo skipping

# remove phones with Display Size lower than 2.5 inches (watch )

for brand in handler.data:
    for phone in handler.data[brand]:
        # check if Misc is present
        if 'Battery' not in handler.data[brand][phone]['specifications']:
            handler.data[brand][phone]['specifications']['Battery'] = {'Type': ['Unknown']}
            continue

        if 'Type' not in handler.data[brand][phone]['specifications']['Display']:
            handler.data[brand][phone]['specifications']['Battery']['Type'] = ['Unknown']
            continue

        handler.data[brand][phone]['specifications']['Battery']['Type'] = handler.data[brand][phone]['specifications']['Battery']['Type'][0].split(', ')

handler.save_data()

Couldn't find program: 'echo'


In [7]:
# get the phones announced after 2019 do not include the ones with no date -> []
latest_phones = {}
for i in handler.data:
    for j in handler.data[i]:
        if handler.data[i][j]['specifications']['Launch']['Announced'] and int(handler.data[i][j]['specifications']['Launch']['Announced'][0]) >= 2019:
            if latest_phones.get(i) is None:
                latest_phones[i] = {}
            latest_phones[i][j] = handler.data[i][j]

sum_of_phones = sum([len(x) for x in latest_phones.values()])
sum_of_phones

3246

In [12]:
# get the Display Size of phones lower than 2.5 inches 
non_phone = {}
for i in latest_phones:
    for j in latest_phones[i]:
        if (float(handler.data[i][j]['specifications']['Display']['Size'][0].split()[0]) < 2.25 and
            handler.data[i][j]['specifications']['Comms']['USB'][0] == 'No') or \
            float(handler.data[i][j]['specifications']['Display']['Size'][0].split()[0]) >= 9:
            
            if non_phone.get(i) is None:
                non_phone[i] = {}
            non_phone[i][j] = latest_phones[i][j]
            # print(handler.data[i][j]['specifications']['Display']['Size'][0], j)

# remove from the latest phones
for i in non_phone:
    for j in non_phone[i]:
        latest_phones[i].pop(j)

# get the models with (pad, tab, watch, fire_hd) in their names
non_phone_2 = {}
for i in latest_phones:
    for j in latest_phones[i]:
        if ('pad' in j.lower() and 'coolpad' not in j.lower()) or 'tab' in j.lower() or 'watch' in j.lower() or 'fire_hd' in j.lower():
            if non_phone_2.get(i) is None:
                non_phone_2[i] = {}
            non_phone_2[i][j] = latest_phones[i][j]

# remove from the latest phones
for i in non_phone_2:
    for j in non_phone_2[i]:
        latest_phones[i].pop(j)
        

# get the models that are Coming Soon
coming_phones = {}
for i in latest_phones:
    for j in latest_phones[i]:
        if latest_phones[i][j]['specifications']['Launch']['Status'][0].lower().find("coming soon") != -1:
            if coming_phones.get(i) is None:
                coming_phones[i] = {}
            coming_phones[i][j] = latest_phones[i][j]

# remove from the latest phones
for i in coming_phones:
    for j in coming_phones[i]:
        latest_phones[i].pop(j)


with open('latest_phones.json', 'w') as f:
    json.dump(latest_phones, f, indent=1, sort_keys=True)

with open('latest_phones.json', 'r') as f:
    latest_phones = json.load(f)

sum([len(x) for x in latest_phones.values()]), sum([len(x) for x in non_phone_2.values()]), sum([len(x) for x in non_phone.values()]), sum([len(x) for x in coming_phones.values()])

(2855, 39, 344, 8)

In [17]:
%%script echo skipping

import bs4
import datetime

device_list = r"https://phonedb.net/sitemap/"


#get the xml file from the sitemap

sitemap = rq.get(device_list)
soup = bs4.BeautifulSoup(sitemap.text, "xml")
# get only devices from the sitemap which is 2017-12-30T22:10:18+01:00 and newer using the lastmod tag

locs = []
for url in soup.findAll("url"):
    # parse the lastmod tag to a string and compare it to the date
    date = datetime.datetime.strptime(url.lastmod.text, "%Y-%m-%dT%H:%M:%S%z")
    if date > datetime.datetime(2017, 12, 30, 22, 0, 0, 0, datetime.timezone.utc) and url.loc.text.startswith("https://phonedb.net/index.php?m=device&id="):
            locs.append(url.loc.text)


#remove the first 2 urls as they are not devices
locs.pop(0)
locs.pop(0)

print("There are " + str(len(locs)) + " devices in phonedb.net")

# find in the xml file the devices that are in latest_phones
phones = {}
# l_p = {'Allview':latest_phones['Allview']}
for brand in latest_phones:
    for phone in latest_phones[brand]:
        models = latest_phones[brand][phone]['specifications']['Misc']['Models']
        for model in models:
            for loc in locs:
                phone_name = latest_phones[brand][phone]['phone_name'].replace(' ', '_')
                if 'Unknown' in latest_phones[brand][phone]['specifications']['Misc']['Models'] and f'{brand}'.lower() in loc and f'_{phone_name}_'.lower() in loc:
                    
                    if phones.get(brand) is None:
                        phones[brand] = {}
                    if phones[brand].get(phone) is None:
                        phones[brand][phone] = [loc]
                    else:
                        phones[brand][phone].append(loc)
                    
                elif f'{brand}'.lower() in loc and f'_{model}'.lower() in loc:
                    if phones.get(brand) is None:
                        phones[brand] = {}
                    if phones[brand].get(phone) is None:
                        phones[brand][phone] = [loc]
                    else:
                        phones[brand][phone].append(loc)     
                    
                elif f'{brand}'.lower() in loc and f'_{phone_name}_'.lower() in loc:
                    if phones.get(brand) is None:
                        phones[brand] = {}
                    if phones[brand].get(phone) is None:
                        phones[brand][phone] = [loc]
                    else:
                        phones[brand][phone].append(loc)   
            


sum([len(x) for x in phones.values()])

UsageError: Cell magic `%%` not found.


In [13]:
sum([len(x) for x in latest_phones.values()])

2855

In [9]:
# print all the latest phones 

for brand in latest_phones:
    for phone in latest_phones[brand]:
        print(brand, latest_phones[brand][phone]['phone_name'])

Allview P10 Life
Allview P10 Max
Allview P10 Mini
Allview P10 Pro
Allview Soul X6 Mini
Allview Soul X6 Xtreme
Allview Soul X7 Pro
Allview Soul X7 Style
Allview V4 Viper
Allview V4 Viper Pro
Allview Viva 1003G
Allview Viva 1003G Lite
Allview Viva 803G
Allview Viva C703
Allview Viva Home
Amazon Fire HD 10 (2019)
Amazon Fire HD 10 (2021)
Amazon Fire HD 10 Plus (2021)
Amazon Fire HD 8 (2020)
Amazon Fire HD 8 (2022)
Amazon Fire HD 8 Plus (2020)
Amazon Fire HD 8 Plus (2022)
Amazon Fire Max 11
Apple iPad (2022)
Apple iPad 10.2 (2019)
Apple iPad 10.2 (2020)
Apple iPad 10.2 (2021)
Apple iPad Air (2019)
Apple iPad Air (2020)
Apple iPad Air (2022)
Apple iPad mini (2019)
Apple iPad mini (2021)
Apple iPad Pro 11 (2020)
Apple iPad Pro 11 (2021)
Apple iPad Pro 11 (2022)
Apple iPad Pro 12.9 (2020)
Apple iPad Pro 12.9 (2021)
Apple iPad Pro 12.9 (2022)
Apple iPhone 11
Apple iPhone 11 Pro
Apple iPhone 11 Pro Max
Apple iPhone 12
Apple iPhone 12 mini
Apple iPhone 12 Pro
Apple iPhone 12 Pro Max
Apple iPhone

In [ ]:


print("Processing Lazada entries...")

# non phone keywords
null_phone =[
    'tempered',
    'case',
    'protector',
    'cover',
    'frame',
    'replacement'
]

data_out = None

# check if file present
if not os.path.isfile('lazada_out.json'):
    with open('lazada_out.json', 'w') as file:
        json.dump({}, file)


with open('lazada_out.json', 'r') as file:
    data_out = json_repair.loads(file.read())


for b in latest_phones:
    print(f"Processing {b}...")
    for p in latest_phones[b]:
        print(f"Processing {latest_phones[b][p]['phone_name']}...")

        # random sleep 2 - 3 seconds
        time.sleep(random.randint(3230, 4000)/1000)
        
        # get the current phone name
        cur_phone = b + " " +  latest_phones[b][p]['phone_name']
        print('[INFO]: Processing ' + cur_phone + '...')

        lazada = f"https://www.lazada.com.ph/shop-mobiles/?ajax=true&q={quote(cur_phone)}&service=official"


        # get the xhtml of the page
        soup = bs4.BeautifulSoup(
            rq.get(
                lazada, headers = {
                    'authority': 'ms-mt--api-web.spain.advgo.net',
                    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
                    'accept': 'application/json, text/plain, */*',
                    'x-adevinta-channel': 'web-desktop',
                    'x-schibsted-tenant': 'coches',
                    'sec-ch-ua-mobile': '?0',
                    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
                    'content-type': 'application/json;charset=UTF-8',
                    'origin': 'https://www.coches.net',
                    'sec-fetch-site': 'cross-site',
                    'sec-fetch-mode': 'cors',
                    'sec-fetch-dest': 'empty',
                    'referer': 'https://www.coches.net/',
                    'accept-language': 'en-US,en;q=0.9,es;q=0.8',
                }).text, 'html.parser')
        
        print(json_repair.loads(soup.text))
        

        # some of the pages has broken json, so we need to repair it
        json_laz = json_repair.loads(soup.text)

        if json_laz.get('mods') is None:
            data_out = {}
            print('[ERROR]: Blocked in Lazada.')
            continue

        if len(data_out):
            for item in data_out: 
                if any([word.lower() in data_out['name'].lower() for word in cur_phone.split()]):
                    if latest_phones[b][p].get('Lazada') is None:

                        if item.get('price') is None:
                            continue
                            item['price'] = 'Unknown'
                        if item.get('originalPrice') is None:
                            continue
                            item['originalPrice'] = item['price']
                        if item.get('review') is None:
                            continue
                            item['review'] = 'Unknown'
                        if item.get('ratingScore') is None:
                            continue
                            item['ratingScore'] = 'Unknown'
                    if item.get('itemUrl') is None:
                        continue
                        item['itemUrl'] = 'Unknown'

                    
                    if float(item['price']) < 1000:
                        # smartphone accessories case
                        continue

                    data_out[cur_phone] = {
                        'URL': item['itemUrl'], 
                        'Ratings': item['ratingScore'],
                        'Reviews': item['review'],
                        'Price': item['price'], 
                        'Original Price': item['originalPrice']
                        }
            
                print('[INFO]: ' + cur_phone + ' is in Lazada.')
                break
            else:
                data_out[cur_phone] = {}
                print('[INFO]: ' + cur_phone + ' is not in Lazada.')

        
        else:
            data_out[cur_phone] = {}
            print('[INFO]: ' + cur_phone + ' is not in Lazada.')    
                            
        with open('lazada_out.json', 'w') as file:
            json.dump(data_out, file, indent=1, sort_keys=True)
    

In [22]:
# %%script echo skipping
#process the data

#load the data
with open('lazada_out.json', 'r') as f:
    latest_phones_lazada = json_repair.loads(f.read())

print(f'Processed {len(latest_phones_lazada)} out of {sum([len(x) for x in latest_phones.values()])} phones in Lazada.')
print(f'Total Phones Available: {len([x for x in latest_phones_lazada if latest_phones_lazada[x] != {}])}')
latest_phones_lst = [b + " " + latest_phones[b][p]['phone_name'] for b in latest_phones for p in latest_phones[b]]
print(f'There are {x if (x:=set(latest_phones_lst) - set(latest_phones_lazada)) else "no"} missing phones.')

# combine the data
latest_phones_combined = latest_phones.copy()
for b in latest_phones_combined:
    for p in latest_phones_combined[b]:
        if latest_phones_lazada.get(b + " " + latest_phones_combined[b][p]['phone_name']):
            latest_phones_combined[b][p]['Lazada'] = latest_phones_lazada[b + " " + latest_phones_combined[b][p]['phone_name']]

# save the data
with open('latest_phones_combined.json', 'w') as f:
    json.dump(latest_phones_combined, f, indent=1, sort_keys=True)


Processed 2855 out of 2855 phones in Lazada.
Total Phones Available: 884
There are no missing phones.


In [93]:
# open the file
with open('latest_phones_combined.json', 'r') as f:
    latest_phones_combined = json_repair.loads(f.read())

# get the sum of the phones

prev = []
for brand in latest_phones:
    for phone in latest_phones[brand]:
        prev.append(phone)

now = []
for brand in latest_phones_combined:
    for phone in latest_phones_combined[brand]:
        now.append(phone)


set(prev) - set(now)


set()

In [94]:
# flatten the data
latest_phones_combined_flat = {}
for b in latest_phones_combined:
    for p in latest_phones_combined[b]:
        latest_phones_combined_flat[p] = latest_phones_combined[b][p]

# remove the data with no Lazada entries
latest_phones_combined_flat = {k: v for k, v in latest_phones_combined_flat.items() if v.get('Lazada')}



In [95]:
%%script echo skipping
# count the build

['glass', 'aluminum', 'plastic', 'unknown', 'titanium', 'ceramic', 'stainless steel', 'carbon fiber']


ll = []
for p in latest_phones_combined_flat:
    if latest_phones_combined_flat[p]['specifications']['Body'].get('Build'):
        print(latest_phones_combined_flat[p]['specifications']['Body']['Build'][0])
        ll.append(latest_phones_combined_flat[p]['specifications']['Body']['Build'][0])
    else:
        print('Unknown')
        ll.append('Unknown')

print(*[x[0] for x in get_top_words(ll, 50)], sep='\n')


Couldn't find program: 'echo'


In [96]:
def get_word_frequencies(lst, words):
    word_freq = {}
    for i, row in enumerate(lst):
        text = row.lower()  # Convert text to lowercase
        for word in text.split():
            if all(v.lower() in text for v in word):  # Convert word to lowercase
                if word in word_freq.keys():
                    word_freq[word].add(i)
                else:
                    word_freq[word] = set()            
    return word_freq

def get_top_words(lst, top_n=10):
    word_count = {}
    for sentence in lst:
        if sentence is None:
            continue
        words = sentence.lower().split()  # Convert text to lowercase
        for word in words:
            word = word.strip('.,!?()[]{}"\'')  # Remove punctuation (optional, depending on your requirements)
            word_count[word] = word_count.get(word, 0) + 1
    top_words = sorted(word_count.items(), key=lambda x: x[1], reverse=True)[:top_n]
    return top_words


In [112]:
# new clean the data to store in dataframe
import pandas as pd
import numpy as np

clean_data = pd.DataFrame()

# add rows as 'phone name key' to the dataframe and column as the 'os'
clean_data = pd.DataFrame(
    index=latest_phones_combined_flat.keys(), 
    columns=[
        'brand',
        'colors',
        'battery_size',
        'battery_endurance',
        'dimensions',
        'weight',
        'os',
        'release',
        'screen_size',
        'screen_resolution',
        'screen_type',
        'camera_count_back',
        'camera_count_front',
        'camera_list_resolution_back',
        'camera_list_resolution_front',
        'camera_resolution_back_recording',
        'camera_resolution_front_recording',
        'chipset',
        'benchmark_antutu',
        'benchmark_geekbench',
        'gpu',
        'ram_min',
        'ram_max',
        'storage_min',
        'storage_max',
        'storage_type',
        'price_min',
        'price_max',
        'lazada_ratings',
        'lazada_reviews',
        'build_material',
        'has_wifi',
        'has_bluetooth',
        'has_cardslot',
    ])

x = []


# add brand
clean_data['brand'] = [latest_phones_combined_flat[p]['brand'] for p in latest_phones_combined_flat]

# add colors
clean_data['colors'] = [latest_phones_combined_flat[p]['specifications']['Misc']['Colors'][0].replace(', ', '|') if latest_phones_combined_flat[p]['specifications']['Misc'].get('Colors') else 'Unknown' for p in latest_phones_combined_flat]

# add battery size
find_battery = re.compile(r"(\d+) mAh", re.IGNORECASE)
# get average battery size
average_battery_size = np.mean([float(find_battery.findall(latest_phones_combined_flat[p]['specifications']['Battery']['Type'][0])[0]) for p in latest_phones_combined_flat if latest_phones_combined_flat[p]['specifications']['Battery'].get('Type') ])
clean_data['battery_size'] = [
    int(
        find_battery.findall(latest_phones_combined_flat[p]['specifications']['Battery']['Type'][0])[0])
        if 
            latest_phones_combined_flat[p]['specifications']['Battery'].get('Type') 
        else 
            np.nan 
        for p in latest_phones_combined_flat]


# add battery endurance
find_b_endurance = re.compile(r"(\d+)h", re.IGNORECASE)
# get average battery endurance
average_battery_endurance = np.mean([float(find_b_endurance.findall(latest_phones_combined_flat[p]['specifications']['Tests']['Battery (old)'][0])[0]) for p in latest_phones_combined_flat  if latest_phones_combined_flat[p]['specifications'].get('Tests') and latest_phones_combined_flat[p]['specifications']['Tests'].get('Battery (old)') ])
clean_data['battery_endurance'] = [
    float(
        find_b_endurance.findall(latest_phones_combined_flat[p]['specifications']['Tests']['Battery (old)'][0])[0]) 
        if 
            latest_phones_combined_flat[p]['specifications'].get('Tests') and latest_phones_combined_flat[p]['specifications']['Tests'].get('Battery (old)')  
        else 
            (average_battery_endurance/average_battery_size)*float(find_battery.findall(latest_phones_combined_flat[p]['specifications']['Battery']['Type'][0])[0])
        for p in latest_phones_combined_flat]

# add dimensions
find_dimensions = re.compile(r"(\d+\.?\d)", re.IGNORECASE)
clean_data['dimensions'] = [
    x[:3]
    if 
        latest_phones_combined_flat[p]['specifications']['Body'].get('Dimensions') and len(x := find_dimensions.findall(latest_phones_combined_flat[p]['specifications']['Body']['Dimensions'][0])) <= 3
    else 
        [np.nan, np.nan, np.nan] 
    for p in latest_phones_combined_flat]

# add weight
find_weight = re.compile(r"(\d+\.?\d+) g", re.IGNORECASE)
clean_data['weight'] = [
    float(x[0])
    if 
        latest_phones_combined_flat[p]['specifications']['Body'].get('Weight') and (x := find_weight.findall(latest_phones_combined_flat[p]['specifications']['Body']['Weight'][0])) 
    else 
        np.nan 
    for p in latest_phones_combined_flat]

# supply missing values
clean_data['weight'] = clean_data['weight'].fillna(clean_data['weight'].mean())

# add os
find = re.compile(r"Android|iOS|Windows|Blackberry|Symbian|Bada|Tizen|KaiOS|Feature phone|HarmonyOS|EMUI", re.IGNORECASE)
clean_data['os'] = ['Android' if (r := find.match(latest_phones_combined_flat[p]['os'])[0]).lower() == 'emui' else r for p in latest_phones_combined_flat]

# add release
clean_data['release'] = [latest_phones_combined_flat[p]['specifications']['Launch']['Announced'][0] if latest_phones_combined_flat[p]['specifications']['Launch'].get('Announced') else None for p in latest_phones_combined_flat]

# add screen size
find_screen_size = re.compile(r"(\d+\.?\d+) inches", re.IGNORECASE)
clean_data['screen_size'] = [
    float(x[0]) 
    if 
        latest_phones_combined_flat[p]['specifications']['Display'].get('Size') and (x := find_screen_size.findall(latest_phones_combined_flat[p]['specifications']['Display']['Size'][0])) 
    else 
        np.nan 
    for p in latest_phones_combined_flat]

# add screen resolution
find_screen_resolution = re.compile(r"(\d+) x (\d+)", re.IGNORECASE)

clean_data['screen_resolution'] = [
    float(x[0][0]) * float(x[0][1])
    if 
        latest_phones_combined_flat[p]['specifications']['Display'].get('Resolution') and len(x := find_screen_resolution.findall(latest_phones_combined_flat[p]['specifications']['Display']['Resolution'][0])) == 1
    else 
        np.nan 
    for p in latest_phones_combined_flat]

# add screen type
find_screen_type = re.compile(r"(\b(?:LCD|OLED|AMOLED|IPS|TFT|P-OLED|Super AMOLED|Super Retina XDR OLED)\b)", re.IGNORECASE)
clean_data['screen_type'] = [
    x[0]
    if 
        latest_phones_combined_flat[p]['specifications']['Display'].get('Type') and (x := find_screen_type.findall(latest_phones_combined_flat[p]['specifications']['Display']['Type'][0])) 
    else 
        'Unknown' 
    for p in latest_phones_combined_flat]

find_camera_resolution = re.compile(r"(\d+\.?\d) MP", re.IGNORECASE)


# add camera count, if Main Camera has key 'Quad' or 'Triple' or 'Dual' or 'Single
clean_data[['camera_count_back', 'camera_list_resolution_back']] = [
    (  
        4, 
        '|'.join(find_camera_resolution.findall(x['Quad'][0])[:4])
     ) if (x := latest_phones_combined_flat[p]['specifications'].get('Main Camera')) and 'Quad' in x 
    else (
        3,
        '|'.join(find_camera_resolution.findall(x['Triple'][0])[:3])
        ) if x and 'Triple' in x 
    else (
        2,
        '|'.join(find_camera_resolution.findall(x['Dual'][0])[:2])
        ) if x and 'Dual' in x 
    else (
        1,
        '|'.join(find_camera_resolution.findall(x['Single'][0])[:1])
          ) if x and 'Single' in x 
    else (
        0,
        '0'
        ) for p in latest_phones_combined_flat]

# add camera count, if Front Camera has key 'Quad' or 'Triple' or 'Dual' or 'Single
clean_data[['camera_count_front', 'camera_list_resolution_front']] = [
    (  
        4, 
        '|'.join(find_camera_resolution.findall(x['Quad'][0])[:4])
     ) if (x := latest_phones_combined_flat[p]['specifications'].get('Selfie camera')) and 'Quad' in x 
    else (
        3,
        '|'.join(find_camera_resolution.findall(x['Triple'][0])[:3])
        ) if x and 'Triple' in x 
    else (
        2,
        '|'.join(find_camera_resolution.findall(x['Dual'][0])[:2])
        ) if x and 'Dual' in x 
    else (
        1,
        '|'.join(find_camera_resolution.findall(x['Single'][0])[:1])
          ) if x and 'Single' in x 
    else (
        0,
        '0'
        ) for p in latest_phones_combined_flat]

# add chipset
clean_data['chipset'] = [
    x[0].split('(')[0].strip() 
    if 
        latest_phones_combined_flat[p]['specifications'].get('Platform') and 
        (x := latest_phones_combined_flat[p]['specifications']['Platform'].get('Chipset'))
    else 
        None 
    for p in latest_phones_combined_flat]


# add benchmark antutu
find_benchmark_antutu = re.compile(r"AnTuTu: (\d+)", re.IGNORECASE)
clean_data['benchmark_antutu'] = [
    int(x[0]) 
    if 
        latest_phones_combined_flat[p]['specifications'].get('Tests') and 
        latest_phones_combined_flat[p]['specifications']['Tests'].get('Performance') and
        (x := find_benchmark_antutu.findall(latest_phones_combined_flat[p]['specifications']['Tests']['Performance'][0]))
    else 
        np.nan 
    for p in latest_phones_combined_flat]


# Group the similar chipsets. Get the average of the benchmark, combining the similar chipsets
for chipset in clean_data['chipset'].value_counts().index:
    # fill only the null values
    clean_data.loc[(clean_data['chipset'] == chipset) , 'benchmark_antutu'] = clean_data.loc[(clean_data['chipset'] == chipset), 'benchmark_antutu'].fillna(clean_data.loc[clean_data['chipset'] == chipset, 'benchmark_antutu'].mean())

# fill the missing values
clean_data['benchmark_antutu'] = clean_data['benchmark_antutu'].fillna(clean_data['benchmark_antutu'].mean()).astype(int)


# add benchmark geekbench
find_benchmark_geekbench = re.compile(r"GeekBench: (\d+)", re.IGNORECASE)
clean_data['benchmark_geekbench'] = [
    int(x[0]) 
    if 
        latest_phones_combined_flat[p]['specifications'].get('Tests') and 
        latest_phones_combined_flat[p]['specifications']['Tests'].get('Performance') and
        (x := find_benchmark_geekbench.findall(latest_phones_combined_flat[p]['specifications']['Tests']['Performance'][0]))
    else 
        np.nan 
    for p in latest_phones_combined_flat]

# Group the similar chipsets. Get the average of the benchmark, combining the similar chipsets
for chipset in clean_data['chipset'].value_counts().index:
    # fill only the null values
    clean_data.loc[(clean_data['chipset'] == chipset) , 'benchmark_geekbench'] = clean_data.loc[(clean_data['chipset'] == chipset), 'benchmark_geekbench'].fillna(clean_data.loc[clean_data['chipset'] == chipset, 'benchmark_geekbench'].mean())

# fill the missing values
clean_data['benchmark_geekbench'] = clean_data['benchmark_geekbench'].fillna(clean_data['benchmark_geekbench'].mean()).astype(int)

# add gpu
clean_data['gpu'] = [
    x[0].split('(')[0].strip() 
    if 
        latest_phones_combined_flat[p]['specifications'].get('Platform') and 
        (x := latest_phones_combined_flat[p]['specifications']['Platform'].get('GPU'))
    else 
        None 
    for p in latest_phones_combined_flat]

# supply missing values from the non missing gpu, get the data from the same chipset
for gpu in clean_data[~clean_data['gpu'].isnull()]['gpu'].value_counts().index:
    # fill only the null values
    clean_data.loc[(clean_data['gpu'] == gpu) , 'chipset'] = clean_data.loc[(clean_data['gpu'] == gpu), 'chipset'].fillna(clean_data.loc[clean_data['gpu'] == gpu, 'chipset'].mode()[0])


# add values for the min and max of ram and storage
find_ram = re.compile(r"(\d+)GB (\d+)GB RAM", re.IGNORECASE)
clean_data[['storage_min', 'storage_max', 'ram_min', 'ram_max']] = [
    (  
        min([tuple(int(x) for x in tup) for tup in zip(*x)][0]),
        max([tuple(int(x) for x in tup) for tup in zip(*x)][0]),
        min([tuple(int(x) for x in tup) for tup in zip(*x)][1]),
        max([tuple(int(x) for x in tup) for tup in zip(*x)][1])
     ) if (x := find_ram.findall(latest_phones_combined_flat[p]['specifications']['Memory']['Internal'][0])) 
    else (
        np.nan,
        np.nan,
        np.nan,
        np.nan
        ) for p in latest_phones_combined_flat]

# add storage type
find_storage_type = re.compile(r"(\b(?:eMMC|UFS|NVMe)\b)", re.IGNORECASE)
clean_data['storage_type'] = [
    x[0]
    if 
        latest_phones_combined_flat[p]['specifications']['Memory'].get('Internal') and (x := find_storage_type.findall(latest_phones_combined_flat[p]['specifications']['Memory']['Internal'][0])) 
    else 
        'Unknown' 
    for p in latest_phones_combined_flat]

# add lazada ratings
clean_data['lazada_ratings'] = [latest_phones_combined_flat[p]['Lazada']['Ratings'] if latest_phones_combined_flat[p].get('Lazada') else np.nan for p in latest_phones_combined_flat]

# add lazada reviews
clean_data['lazada_reviews'] = [latest_phones_combined_flat[p]['Lazada']['Reviews'] if latest_phones_combined_flat[p].get('Lazada') else 0 for p in latest_phones_combined_flat]

# add build material
find_build_material = re.compile(r"(\b(?:glass|aluminum|plastic|titanium|ceramic|stainless steel|carbon fiber)\b)", re.IGNORECASE)
clean_data['build_material'] = [
    '|'.join(x)
    if 
        latest_phones_combined_flat[p]['specifications']['Body'].get('Build') and (x := find_build_material.findall(latest_phones_combined_flat[p]['specifications']['Body']['Build'][0])) 
    else 
        None 
    for p in latest_phones_combined_flat]

# fill the missing values
clean_data['build_material'] = clean_data['build_material'].fillna(clean_data['build_material'].mode()[0])

# euro to php
def euro_to_php(euro):
    if euro is None or euro == '':
        return None
    return float(euro) * 60.25

# dollar to php
def dollar_to_php(dollar):
    if dollar is None or dollar == '':
        return None
    return float(dollar) * 55.90


# add min and max price
find_price = re.compile(r"(\d+\.?\d) EUR|\$\u2009(\d+(?:\.\d+)?)", re.IGNORECASE)

clean_data[['price_min', 'price_max']] = [
        [min(y := [
            euro_to_php(x[0][0]) or dollar_to_php(x[0][1]), 
            float(latest_phones_combined_flat[p]['Lazada']['Price']),
            float(latest_phones_combined_flat[p]['Lazada']['Original Price'])],
            ),
        max(y)]
    if 
        latest_phones_combined_flat[p]['specifications']['Misc'].get('Price') and (x:= find_price.findall(latest_phones_combined_flat[p]['specifications']['Misc']['Price'][0]))
    else
    
        (np.nan, np.nan)
    for p in latest_phones_combined_flat]

# supply missing values
clean_data['price_min'] = clean_data['price_min'].fillna(clean_data['price_min'].mean())
clean_data['price_max'] = clean_data['price_max'].fillna(clean_data['price_max'].mean())

# add camera resolution back recording
find_camera_resolution_back_recording = re.compile(r"(\d+)fps", re.IGNORECASE)

# add camera resolution front recording
find_camera_resolution_recording = re.compile(r"((?:8K|4K|1080p|720p).*.fps|4K|1080p|720p)", re.IGNORECASE)

ll = []
for p in latest_phones_combined_flat:
    if latest_phones_combined_flat[p]['specifications'].get('Main Camera') and latest_phones_combined_flat[p]['specifications']['Main Camera'].get('Video'):
        o = []
        for i in latest_phones_combined_flat[p]['specifications']['Main Camera']['Video'][0].split(','):
            if (x := find_camera_resolution_recording.findall(i)):
                o.append(x[0])
            elif i.lower().find('yes') != -1:
                o.append('1080p@30fps')

        if o:
            ll.append('|'.join(o))
        else:
            ll.append(None)
    else:
        ll.append(None)
clean_data['camera_resolution_back_recording'] = ll

# add camera resolution front recording
ll = []
for p in latest_phones_combined_flat:
    if latest_phones_combined_flat[p]['specifications'].get('Selfie camera') and latest_phones_combined_flat[p]['specifications']['Selfie camera'].get('Video'):
        o = []
        for i in latest_phones_combined_flat[p]['specifications']['Selfie camera']['Video'][0].split(','):
            if (x := find_camera_resolution_recording.findall(i)):
                o.append(x[0])
            elif i.lower().find('yes') != -1:
                o.append('xxxxx@xxxxx')

        if o:
            ll.append('|'.join(o))
        else:
            ll.append(None)
    else:
        ll.append(None)
clean_data['camera_resolution_front_recording'] = ll

# add has wireless
find_wifi = re.compile(r".*\|.*", re.IGNORECASE)
ll = []
for p in latest_phones_combined_flat:
    if latest_phones_combined_flat[p]['specifications'].get('Comms') and latest_phones_combined_flat[p]['specifications']['Comms'].get('WLAN'):
        done = False
        if (x := latest_phones_combined_flat[p]['specifications']['Comms']['WLAN'][0].split(',')[0]).lower() != 'yes':
            for i in x.split():
                if find_wifi.match(i := i.replace('/', '|')):
                    ll.append(i)
                    done = True
                    break

            if not done:
                ll.append(None)
        else:
            ll.append(None)
    else:
        ll.append('No')
clean_data['has_wifi'] = ll

# supply missing values
clean_data['has_wifi'] = clean_data['has_wifi'].fillna(clean_data['has_wifi'].mode()[0])

# add has bluetooth
ll = []
for p in latest_phones_combined_flat:
    if latest_phones_combined_flat[p]['specifications'].get('Comms') and latest_phones_combined_flat[p]['specifications']['Comms'].get('Bluetooth'):
        if (x:=latest_phones_combined_flat[p]['specifications']['Comms']['Bluetooth'][0].split(',')[0]).lower() != 'no' and x != 'yes':
            ll.append(x)
        elif x.lower() == 'yes':
            ll.append(None)
        else:
            ll.append('No')
    else:
        ll.append('No')
clean_data['has_bluetooth'] = ['Yes' if (x := latest_phones_combined_flat[p]['specifications']['Comms'].get('Bluetooth')) and x[0].lower() != 'no' else 'No' for p in latest_phones_combined_flat]

# supply missing values
clean_data['has_bluetooth'] = clean_data['has_bluetooth'].fillna(clean_data['has_bluetooth'].mode()[0])

# add has cardslot
clean_data['has_cardslot'] = ['Yes' if (x := latest_phones_combined_flat[p]['specifications']['Memory'].get('Card slot')) and x[0].lower() != 'no' else 'No' for p in latest_phones_combined_flat]

# remove the data with null values
clean_data = clean_data.dropna()

clean_data[['dimensions', 'screen_resolution']]


,dimensions,screen_resolution
apple_iphone_11-9848,"[nan, nan, nan]",1483776.0
apple_iphone_11_pro-9847,"[nan, nan, nan]",2740500.0
apple_iphone_11_pro_max-9846,"[nan, nan, nan]",3338496.0
apple_iphone_12-10509,"[nan, nan, nan]",2962440.0
apple_iphone_12_mini-10510,"[nan, nan, nan]",2527200.0
...,...,...
vivo_y51a-10684,"[nan, nan, nan]",2600640.0
vivo_y55-11744,"[nan, nan, nan]",2592000.0
vivo_y73s-10504,"[nan, nan, nan]",2592000.0
vivo_y76_5g-11231,"[nan, nan, nan]",2600640.0


In [113]:
[
    x[:3]
    if 
        latest_phones_combined_flat[p]['specifications']['Body'].get('Dimensions') and len(x := find_dimensions.findall(latest_phones_combined_flat[p]['specifications']['Body']['Dimensions'][0])) <= 3
    else 
        [np.nan, np.nan, np.nan] 
    for p in latest_phones_combined_flat]

[[nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan, nan],
 [nan, nan

In [ ]:
[
      
        [min(y := [
            euro_to_php(x[0][0]) or dollar_to_php(x[0][1]), 
            float(latest_phones_combined_flat[p]['Lazada']['Price']),
            float(latest_phones_combined_flat[p]['Lazada']['Original Price'])],
            ),
        max(y)]
    if 
        latest_phones_combined_flat[p]['specifications']['Misc'].get('Price') and (x:= find_price.findall(latest_phones_combined_flat[p]['specifications']['Misc']['Price'][0]))
    else
    
        (np.nan, np.nan)
    for p in latest_phones_combined_flat]

[[12745.199999999999, 37990.0],
 [16155.1, 34990.0],
 [18600.725, 43460.0],
 [13886.677999999998, 53990.0],
 [13807.3, 25990.0],
 [21571.809999999998, 44990.0],
 [24917.425, 56180.0],
 [21687.523, 52990.0],
 [23464.024999999998, 52490.0],
 [28786.823, 68990.0],
 [34375.705, 76990.0],
 [29462.094999999998, 52990.0],
 [29627.0, 68990.0],
 [35755.876, 64990.0],
 [45732.907999999996, 71990.0],
 [39098.136999999995, 61990.0],
 [48945.481, 76990.0],
 [55.9, 91990.0],
 [7258.056, 16990.0],
 [7265.322999999999, 29990.0],
 [32925.1, 36999.0],
 [29000.0, 62057.5],
 [29569.982, 80000.0],
 [46113.0, 90375.0],
 [36334.441, 65995.0],
 [111.8, 77995.0],
 [31999.0, 36334.441],
 [111.8, 15000.0],
 [38542.491, 50995.0],
 [32925.1, 69995.0],
 (nan, nan),
 [35216.441, 70000.0],
 [8762.0, 10845.0],
 [12018.5, 19999.0],
 [5499.0, 7230.0],
 (nan, nan),
 [7090.0, 14999.0],
 [4399.0, 11447.5],
 [9726.041000000001, 15546.0],
 [13129.0, 19588.0],
 [21999.0, 40000.0],
 (nan, nan),
 [17825.0, 32589.0],
 [18075.0, 

In [103]:
ll = []
for p in latest_phones_combined_flat:
    if latest_phones_combined_flat[p]['specifications'].get('Comms') and latest_phones_combined_flat[p]['specifications']['Comms'].get('Bluetooth'):
        if (x:=latest_phones_combined_flat[p]['specifications']['Comms']['Bluetooth'][0].split(',')[0]).lower() != 'no' and x != 'yes':
            ll.append(x)
        elif x.lower() == 'yes':
            ll.append(None)
        else:
            ll.append('No')
    else:
        ll.append('No')
        

['5.0, A2DP, LE'] apple_iphone_11-9848
['5.0, A2DP, LE'] apple_iphone_11_pro-9847
['5.0, A2DP, LE'] apple_iphone_11_pro_max-9846
['5.0, A2DP, LE'] apple_iphone_12-10509
['5.0, A2DP, LE'] apple_iphone_12_mini-10510
['5.0, A2DP, LE'] apple_iphone_12_pro-10508
['5.0, A2DP, LE'] apple_iphone_12_pro_max-10237
['5.0, A2DP, LE'] apple_iphone_13-11103
['5.0, A2DP, LE'] apple_iphone_13_mini-11104
['5.0, A2DP, LE'] apple_iphone_13_pro-11102
['5.0, A2DP, LE'] apple_iphone_13_pro_max-11089
['5.3, A2DP, LE'] apple_iphone_14-11861
['5.3, A2DP, LE'] apple_iphone_14_plus-11862
['5.3, A2DP, LE'] apple_iphone_14_pro-11860
['5.3, A2DP, LE'] apple_iphone_14_pro_max-11773
['5.3, A2DP, LE'] apple_iphone_15-12559
['5.3, A2DP, LE'] apple_iphone_15_pro-12557
['5.3, A2DP, LE'] apple_iphone_15_pro_max-12548
['5.0, A2DP, LE'] apple_iphone_se_(2020)-10170
['5.0, A2DP, LE'] apple_iphone_se_(2022)-11410
['5.2, A2DP, LE, aptX HD, aptX Adaptive'] asus_rog_phone_5-10715
['5.2, A2DP, LE, aptX HD, aptX Adaptive'] asus_ro

In [ ]:
find_price = re.compile(r"(\d+\.?\d) EUR|\$\u2009(\d+(?:\.\d+)?)", re.IGNORECASE)

[
        x 
    if 
        (x := find_camera_resolution_recording.findall(i)) 
    else 
        '1080p@30fps' 
        if 
            i.lower().find('yes') != -1 
        else 
            None 
    for p in latest_phones_combined_flat 
    if 
        latest_phones_combined_flat[p]['specifications'].get('Main Camera') and 
        latest_phones_combined_flat[p]['specifications']['Main Camera'].get('Video') 
    for i in latest_phones_combined_flat[p]['specifications']['Main Camera']['Video'][0].split(',')]


[['4K@24/30/60fps'],
 ['1080p@30/60/120/240fps'],
 None,
 None,
 ['4K@24/30/60fps'],
 ['1080p@30/60/120/240fps'],
 None,
 None,
 ['4K@24/30/60fps'],
 ['1080p@30/60/120/240fps'],
 None,
 None,
 ['4K@24/30/60fps'],
 ['1080p@30/60/120/240fps'],
 None,
 None,
 None,
 ['4K@24/30/60fps'],
 ['1080p@30/60/120/240fps'],
 None,
 None,
 None,
 ['4K@24/30/60fps'],
 ['1080p@30/60/120/240fps'],
 None,
 None,
 None,
 ['4K@24/30/60fps'],
 ['1080p@30/60/120/240fps'],
 None,
 None,
 None,
 ['4K@24/30/60fps'],
 ['1080p@30/60/120/240fps'],
 None,
 None,
 None,
 ['4K@24/30/60fps'],
 ['1080p@30/60/120/240fps'],
 None,
 None,
 None,
 ['4K@24/30/60fps'],
 ['1080p@30/60/120/240fps'],
 None,
 None,
 None,
 ['1080p@30fps'],
 None,
 ['4K@24/30/60fps'],
 ['1080p@30/60/120/240fps'],
 None,
 None,
 None,
 ['1080p@30fps'],
 None,
 ['4K@24/25/30/60fps'],
 ['1080p@25/30/60/120/240fps'],
 None,
 None,
 ['4K@30fps'],
 None,
 ['4K@24/25/30/60fps'],
 ['1080p@25/30/60/120/240fps'],
 None,
 None,
 ['4K@30fps'],
 None,
 ['4K@

In [ ]:
x = []
[
    x := find_dimensions.findall(latest_phones_combined_flat[p]['specifications']['Body']['Dimensions'][0])[:3] 
    if 
        latest_phones_combined_flat[p]['specifications']['Body'].get('Dimensions') and len(x) <= 3
    else 
        [np.nan, np.nan, np.nan] 
    for p in latest_phones_combined_flat
]

[['150.9', '75.7', '8.3'],
 ['144', '71.4', '8.1'],
 ['158', '77.8', '8.1'],
 ['146.7', '71.5', '7.4'],
 ['131.5', '64.2', '7.4'],
 ['146.7', '71.5', '7.4'],
 ['160.8', '78.1', '7.4'],
 ['146.7', '71.5', '7.7'],
 ['131.5', '64.2', '7.7'],
 ['146.7', '71.5', '7.7'],
 ['160.8', '78.1', '7.7'],
 ['146.7', '71.5', '7.8'],
 ['160.8', '78.1', '7.8'],
 ['147.5', '71.5', '7.9'],
 ['160.7', '77.6', '7.9'],
 ['147.6', '71.6', '7.8'],
 ['146.6', '70.6', '8.3'],
 ['159.9', '76.7', '8.3'],
 ['138.4', '67.3', '7.3'],
 ['138.4', '67.3', '7.3'],
 ['172.8', '77.3', '10.3'],
 ['172.8', '77.3', '9.9'],
 ['173', '77', '10.3'],
 ['173', '77', '10.4'],
 ['173', '77', '10.4'],
 ['173', '77', '10.3'],
 ['173', '77', '10.4'],
 ['173', '77', '10.4'],
 ['173', '77', '10.3'],
 ['173', '77', '10.3'],
 ['163.8', '76.8', '8.9'],
 ['146.5', '68.1', '9.4'],
 ['166.3', '77.6', '8.8'],
 ['162.2', '73.6', '8.6'],
 ['156.4', '72.9', '8.9'],
 ['164.6', '76.9', '9.4'],
 ['164.3', '76.1', '8.7'],
 ['163.7', '75.5', '8.6'],
 

In [ ]:
find_camera_resolution.findall(
    latest_phones_combined_flat['apple_iphone_11-9848']['specifications']['Main Camera']['Dual'][0]
)

[]

In [ ]:

find = re.compile(r"(\d+)h", re.IGNORECASE)
[float(find.findall(latest_phones_combined_flat[p]['specifications']['Tests']['Battery (old)'][0])[0]) if latest_phones_combined_flat[p]['specifications'].get('Tests') and latest_phones_combined_flat[p]['specifications']['Tests'].get('Battery (old)')  else np.nan for p in latest_phones_combined_flat]


[94.0,
 86.0,
 102.0,
 84.0,
 69.0,
 81.0,
 95.0,
 89.0,
 75.0,
 85.0,
 121.0,
 90.0,
 102.0,
 86.0,
 121.0,
 98.0,
 86.0,
 118.0,
 59.0,
 62.0,
 110.0,
 nan,
 nan,
 nan,
 nan,
 119.0,
 nan,
 103.0,
 131.0,
 131.0,
 nan,
 109.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 199.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 98.0,
 62.0,
 76.0,
 nan,
 95.0,
 86.0,
 84.0,
 94.0,
 96.0,
 83.0,
 76.0,
 83.0,
 90.0,
 100.0,
 100.0,
 nan,
 nan,
 nan,
 95.0,
 nan,
 nan,
 nan,
 115.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 97.0,
 81.0,
 nan,
 89.0,
 nan,
 nan,
 89.0,
 nan,
 nan,
 n

In [ ]:
latest_phones_combined_flat['apple_iphone_11-9848']

{'brand': 'Apple',
 'dimension': '194g, 8.3mm thickness',
 'os': 'iOS 13, up to iOS 17.3',
 'phone_images': ['https://fdn2.gsmarena.com/vv/pics/apple/apple-iphone-11-1.jpg',
  'https://fdn2.gsmarena.com/vv/pics/apple/apple-iphone-11-3.jpg',
  'https://fdn2.gsmarena.com/vv/pics/apple/apple-iphone-11-2.jpg',
  'https://fdn2.gsmarena.com/vv/pics/apple/apple-iphone-11-9.jpg'],
 'phone_name': 'iPhone 11',
 'release_date': 'Released 2019, September 20',
 'specifications': {'Battery': {'Charging': ['Wired, PD2.0, 50% in 30 min (advertised)\nWireless (Qi)'],
   'Type': ['Li-Ion 3110 mAh']},
  'Body': {'Build': ['Glass front (Corning-made glass), glass back (Corning-made glass), aluminum frame (7000 series)'],
   'Dimensions': ['150.9 x 75.7 x 8.3 mm (5.94 x 2.98 x 0.33 in)'],
   'Other': ['IP68 dust/water resistant (up to 2m for 30 min)\nApple Pay (Visa, MasterCard, AMEX certified)'],
   'SIM': ['Nano-SIM and eSIM or Dual SIM (Nano-SIM, dual stand-by) - for China'],
   'Weight': ['194 g (6.84 

In [ ]:
list(latest_phones_combined_flat.items())[:10]

[
    'Brand',
    'Price',
    'Color',
    'Battery Size',
    'Battery Removavle',
    'Battery Endurance',
    'Dimensions',
    'Weight',
    'OS'
]


[('allview_p10_life-9621',
  {'brand': 'Allview',
   'dimension': '177.6g, 9.4mm thickness',
   'os': 'Android 8.1',
   'phone_images': ['https://fdn2.gsmarena.com/vv/pics/allview/allview-p10-life.jpg'],
   'phone_name': 'P10 Life',
   'release_date': 'Released 2019, January',
   'specifications': {'Battery': {'Type': ['Li-Po 2500 mAh battery']},
    'Body': {'Dimensions': ['149.4 x 71.9 x 9.4 mm (5.88 x 2.83 x 0.37 in)'],
     'SIM': ['Dual SIM (Micro-SIM, dual stand-by)'],
     'Weight': ['177.6 g (6.28 oz)']},
    'Comms': {'Bluetooth': ['4.2, A2DP, LE'],
     'NFC': ['No'],
     'Positioning': ['GPS'],
     'Radio': ['FM radio'],
     'USB': ['microUSB 2.0'],
     'WLAN': ['Wi-Fi 802.11 b/g/n']},
    'Display': {'Resolution': ['480 x 960 pixels, 18:9 ratio (~201 ppi density)'],
     'Size': ['5.34 inches'],
     'Type': ['IPS LCD']},
    'Features': {'Sensors': ['Accelerometer, proximity']},
    'Launch': {'Announced': ['2019', 'January'],
     'Status': ['Available. Released 2019,